In [22]:
import cv2
import numpy as np
from pathlib import Path
import torch
from torchvision import transforms, models
import os
from torchvision.models import ResNet18_Weights

class VideoSegmenter:
   def __init__(self, similarity_threshold=0.85):
       self.model = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
       self.model = torch.nn.Sequential(*list(self.model.children())[:-1])
       self.model.eval()
       self.transform = transforms.Compose([
           transforms.ToPILImage(),
           transforms.Resize((224, 224)),
           transforms.ToTensor(),
           transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                             std=[0.229, 0.224, 0.225])
       ])
       self.similarity_threshold = similarity_threshold

   def extract_features(self, frame):
       frame_tensor = self.transform(frame).unsqueeze(0)
       with torch.no_grad():
           features = self.model(frame_tensor)
       return features.squeeze()

   def compute_similarity(self, feat1, feat2):
       return torch.cosine_similarity(feat1.unsqueeze(0), 
                                    feat2.unsqueeze(0)).item()

   def segment_video(self, video_path, output_dir):
       Path(output_dir).mkdir(parents=True, exist_ok=True)
       
       cap = cv2.VideoCapture(video_path)
       fps = cap.get(cv2.CAP_PROP_FPS)
       frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
       
       segments = []
       current_segment = []
       prev_features = None
       frame_interval = 8
       
       video_name = Path(video_path).stem
       
       for frame_idx in range(0, frame_count, frame_interval):
           cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
           ret, frame = cap.read()
           if not ret:
               break
               
           current_features = self.extract_features(frame)
           
           if prev_features is None:
               current_segment.append(frame_idx)
               prev_features = current_features
               continue
               
           similarity = self.compute_similarity(prev_features, current_features)
           
           if similarity < self.similarity_threshold:
               current_segment.append(frame_idx - frame_interval)
               segments.append(current_segment)
               current_segment = [frame_idx]
               
           prev_features = current_features
           
       if current_segment:
           current_segment.append(frame_idx)
           segments.append(current_segment)
           
       for i, (start, end) in enumerate(segments):
           segment_time_start = start / fps
           segment_time_end = end / fps
           output_filename = f'{video_name}_segment_{i}_{segment_time_start:.2f}-{segment_time_end:.2f}.mp4'
           output_path = os.path.join(output_dir, output_filename)
           self._save_segment(video_path, output_path, start, end, fps)
           
       cap.release()
       return segments

   def _save_segment(self, input_path, output_path, start_frame, end_frame, fps):
       cap = cv2.VideoCapture(input_path)
       cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
       
       fourcc = cv2.VideoWriter_fourcc(*'mp4v')
       out = None
       
       while cap.get(cv2.CAP_PROP_POS_FRAMES) <= end_frame:
           ret, frame = cap.read()
           if not ret:
               break
               
           if out is None:
               height, width = frame.shape[:2]
               out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
               
           out.write(frame)
           
       cap.release()
       if out:
           out.release()

if __name__ == "__main__":
   # For video segmentation
   video_dir = r"C:\Users\admin\film_dataset"
   segments_dir = r"C:\Users\admin\film_dataset\all_segments"
   
   # First segment videos
   segmenter = VideoSegmenter(similarity_threshold=0.85)
   Path(segments_dir).mkdir(parents=True, exist_ok=True)
   
   for video_file in Path(video_dir).glob("*.mp4"):
       segments = segmenter.segment_video(str(video_file), segments_dir)
       print(f"Processed {video_file.name}: {len(segments)} segments created")


Processed segment_1.mp4: 2 segments created
Processed segment_10.mp4: 1 segments created
Processed segment_2.mp4: 2 segments created
Processed segment_3.mp4: 3 segments created
Processed segment_4.mp4: 2 segments created
Processed segment_5.mp4: 2 segments created
Processed segment_6.mp4: 2 segments created
Processed segment_7.mp4: 1 segments created
Processed segment_8.mp4: 2 segments created
Processed segment_9.mp4: 2 segments created


In [23]:
import cv2
import numpy as np
from pathlib import Path
import torch
from torchvision import transforms, models
from sklearn.metrics.pairwise import cosine_similarity
from torchvision.models import ResNet18_Weights

class VideoMatcher:
   def __init__(self):
       self.model = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
       self.model = torch.nn.Sequential(*list(self.model.children())[:-1]) 
       self.model.eval()
       self.transform = transforms.Compose([
           transforms.ToPILImage(),
           transforms.Resize((224, 224)),
           transforms.ToTensor(),
           transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
       ])

   def get_segment_features(self, video_path, sample_rate=8):
       features = []
       cap = cv2.VideoCapture(video_path)
       frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
       
       for frame_idx in range(0, frame_count, sample_rate):
           cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
           ret, frame = cap.read()
           if not ret:
               break
               
           frame_tensor = self.transform(frame).unsqueeze(0)
           with torch.no_grad():
               feat = self.model(frame_tensor)
           features.append(feat.squeeze().numpy())
           
       cap.release()
       return np.mean(features, axis=0)

   def search_segments(self, query_segment, segments_dir, top_k=5):
       query_features = self.get_segment_features(query_segment)
       
       results = []
       for segment_path in Path(segments_dir).glob("segment_*.mp4"):
           segment_features = self.get_segment_features(str(segment_path))
           similarity = cosine_similarity([query_features], [segment_features])[0][0]
           results.append((segment_path, similarity))
           
       results.sort(key=lambda x: x[1], reverse=True)
       return results[:top_k]

query_path = r"C:\Users\admin\segment_8_scene_7_140_3.20-4.27.mp4"
segments_dir = r"C:\Users\admin\film_dataset\all_segments"  

matcher = VideoMatcher()
similar_segments = matcher.search_segments(query_path, segments_dir)

print("\nTop similar segments with full paths:")
for path, similarity in similar_segments:
   print(f"Segment path: {path.absolute()}, Similarity: {similarity:.4f}")


Top similar segments with full paths:
Segment path: C:\Users\admin\film_dataset\all_segments\segment_8_segment_1_2.67-6.13.mp4, Similarity: 0.9860
Segment path: C:\Users\admin\film_dataset\all_segments\segment_9_segment_0_0.00-1.87.mp4, Similarity: 0.9161
Segment path: C:\Users\admin\film_dataset\all_segments\segment_9_segment_1_2.13-6.13.mp4, Similarity: 0.8748
Segment path: C:\Users\admin\film_dataset\all_segments\segment_4_segment_0_0.00-1.60.mp4, Similarity: 0.7882
Segment path: C:\Users\admin\film_dataset\all_segments\segment_2_segment_1_3.20-6.13.mp4, Similarity: 0.7874
